In [ ]:
import glob
import json
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
# import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.2)
sns.set_style("white")
# Smallest context:
sns.set_context("paper")
# matplotlib.rcParams['pdf.fonttype'] = 42
# matplotlib.rcParams['ps.fonttype'] = 42
# plt.rcParams['axes.linewidth'] = 2.0 #set the value globally

agg_names = {
       'mean': 'Mean',
        'krum': 'Krum',
        'geomed': 'GeoMed',
        'autogm': 'AutoGM',
        'median': 'Median',
        'trimmedmean': 'TrimmedMean',
        'centeredclipping': 'CenteredClipping',
        'clustering': 'Clustering', 
        'clippedclustering': 'ClippedClustering',
}

In [ ]:
def read_json(path, agg=None):
    validation = []
    with open(path, "r") as f:
        count = 0
        for line in f:
            line=line.strip().replace("'", '"')
            line = line.replace("nan", '100.0')
            count+=1
            if count % 2 == 0: #this is the remainder operator
#                 continue
                pass
#             ine = line.replace("nan", '"nan"')
            try:
                data = json.loads(line)
            except:
                print(line)
                raise
            if data['_meta']['type'] == 'test':
                validation.append({
                    "Round": data["Round"], 
                    "Accuracy (%)": data['top1'],
                    r"$\beta$": float(re.findall(".*momentum(.*)_bz.*",path)[0]),
                    "Agg": agg_names[agg] if agg else agg_names[path.split("_")[7]],
                    "M": int(float(re.findall(".*b(.*)_attack.*",path)[0])),
#                     "Batch Size": int(path.split("_")[-3][2:]),
                    "Batch Size": int(float(re.findall(".*bz(.*)_local.*",path)[0])),
                    "Loss": data["Loss"],
                    "IID": "Yes" if "noniid" not in path else "No"
                })
    return validation

                    
def transform(entry, atk, agg, m, s, d, b, bz):
    if "large" in atk:
        atk_name = r"IPM ($\epsilon = 100$)"
    elif atk == "IPM":
        atk_name = r"IPM ($\epsilon = 0.5$)"
    else:
        atk_name = atk
        
    if b == 0:
        atk_name = "No Attack"
        
    return {
        'Round Number': entry['E'],
        'Accuracy (%)': entry['top1'],
        'ATK': atk_name,
        "Batch Size": bz,
        'Setting': d.replace('f', 'F', 1),
        'AGG': fedavg_methods[agg]['name'],
        r"$\beta$": m,
        'seed': s,
        r"$M$" : b,
        r"$\delta=0$": False,
    }


In [ ]:
root_dir = "/home/shenghui/blades/scripts/outputsadaptive"
root_dir_noniid = "/home/shenghui/blades/scripts/outputs"
aggs = ['mean', 'krum', 'median', 'trimmedmean',  'centeredclipping', 'clustering', 'clippedclustering']
aggs = ['clustering', 'clippedclustering', 'mean', 'krum',  'geomed', 'autogm', 'median', 'trimmedmean',  'centeredclipping']
# aggs = ['clippedclustering', 'mean', 'krum',  'geomed', 'autogm', 'median', 'trimmedmean',  'centeredclipping']atks = ["ipm", "signflipping", "labelflipping", "alie", "noise"] #* 2
atk_names = ["IPM", "SF", "LF", "ALIE", "Noise"] #* 2

agg = 'clippedclustering'

settings = ['cifar10_fedsgd', 'mnist_fedsgd']#, 'cifar10_fedavg', 'mnist_fedavg']
settings = ['cifar10_fedavg', 'mnist_fedavg']
setting = 'mnist_fedsgd'
# for setting in settings:
def load_data(setting='mnist_fedsgd'):
    history = []
    file_paths = glob.glob(f"{root_dir}/{setting}/*attackclippedclustering*_{agg}_*/stats")
#     print(file_paths)
    for file_path in file_paths:
        agg_validation = read_json(file_path, agg)
        for e in agg_validation:
            e['Dataset'] = "CIFAR10" if "cifar10" in settings else 'MNIST'
            e['Alg.'] = "FedAvg" if "fedavg" in settings else 'FedSGD'
            e['Setting'] = setting
        history.extend(agg_validation)

    file_paths = glob.glob(f"{root_dir_noniid}/{setting}/*attackclippedclustering*_{agg}_*/stats")
#     print(file_paths)
    for file_path in file_paths:
        agg_validation = read_json(file_path, agg)
        for e in agg_validation:
            e['Dataset'] = "CIFAR10" if "cifar10" in settings else 'MNIST'
            e['Alg.'] = "FedAvg" if "fedavg" in settings else 'FedSGD'
            e['Setting'] = setting
        history.extend(agg_validation)

    data = pd.DataFrame(history)
    return data

In [ ]:
def plot_one(data, legend=False, x_max=6000, save_name='figure'):
    g = sns.relplot(
        data=data,
        x="Round", 
        y="Loss", 
    #     y="Accuracy (%)",
        col=r"IID", 
        hue="M", 
        style="M",
    #     hue=r"$\beta$", col="Agg", style=r"$\beta$",
        height=2.5, aspect=1.2,
        # col_order=col_order,
        dashes=True,
    #     markers=True,
    #     col_wrap=4,
        linewidth=1.5,
        kind="line",
        palette="bright",
        legend = legend,
    )
    g.set(yscale="log")
    g.set(ylim=(1e-1, 1e6))
    g.set(xlim=(0, x_max))
    if legend:
        sns.move_legend(g, "upper left", bbox_to_anchor=(.18, 0.9), 
                ncol=1, title="M", frameon=True, columnspacing=0.6)
    g.savefig(f"{save_name}.pdf", bbox_inches = "tight", dpi=500) 

In [ ]:
setting = 'mnist_fedsgd'
data = load_data(setting)
plot_one(data, legend=True, save_name=setting)

In [ ]:
setting = 'cifar10_fedsgd'
data = load_data(setting)
plot_one(data, legend=False, save_name=setting)

In [ ]:
setting = 'cifar10_fedavg'
data = load_data(setting)
plot_one(data, legend=False, x_max=600, save_name=setting)

In [ ]:
setting = 'mnist_fedavg'
data = load_data(setting)
plot_one(data, legend=False, x_max=600, save_name=setting)